In [36]:
import os
import sys;sys.setdefaultencoding('UTF8')
import itertools

import chardet
import json

import time

import pydocumentdb.documents as documents
import pydocumentdb.document_client as document_client
import pydocumentdb.errors as errors
import pydocumentdb.http_constants as http_constants

import pandas as pd; pd.options.display.max_rows = 65; pd.options.display.max_columns = 40
import matplotlib.pyplot as plt
plt.style.use('seaborn')
import numpy as np

import view.config as cfg
import view.storage as st

import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

from IPython.core.display import HTML
from IPython.display import display

In [2]:
HOST = cfg.settings.host
MASTER_KEY = cfg.settings.master_key
DATABASE_ID = cfg.settings.database_id
COLLECTION_ID = cfg.settings.collection_id
COLLECTION_ID = "rte4"

In [3]:
client = document_client.DocumentClient(HOST, {'masterKey': MASTER_KEY})
coll_link = st.GetCollectionLink(client, DATABASE_ID, COLLECTION_ID)

In [4]:
res_names = ["imbalance_data", "insufficients_offers", "prices", "peak_daily_margins", "signals",
             "volumes_per_energy_type","procured_reserves", "accepted_offers"]

In [5]:
proc_id = "ReadResources"
options = { 'enableScriptLogging': True }
params = {
    "resources": res_names,
    "start": "2014-01-01T00:00:00+02:00",
    "end": "2017-12-31T23:30:00+01:00",
    "options" : {
        "pageSize":50,
        "enableScan" : True,
        "continuation": None
    }
}
results = []

while True:
    response = st.ExecuteProcedure( client, coll_link, proc_id, params, options )
    results = results + response["queryResponse"]
    if not "lastContinuationToken" in response.viewkeys() or response["lastContinuationToken"] == None:
        break
    else:
        params["options"]["continuation"] = response["lastContinuationToken"]
            

In [56]:
doo = df.reset_index().head()
doo

,start_date,end_date,available_value,countertrading_xb_redispatching_exported_volume,countertrading_xb_redispatching_imported_volume,deltap_volume,direction,downward_marginal_price,downward_marginal_price_as,downward_marginal_price_margin,downward_volume_afrr,downward_volume_fcr,downward_volume_mfrr,downward_volume_need_rte,downward_volume_rr,downward_weighted_average_price,downward_weighted_average_price_as,downward_weighted_average_price_margin,downward_weighted_average_price_mfrr,downward_weighted_average_price_rr,...,system_trend,tso_mutual_emergency_exported_volume,tso_mutual_emergency_imported_volume,type,upward_marginal_price,upward_marginal_price_as,upward_marginal_price_margin,upward_volume_afrr,upward_volume_fcr,upward_volume_mfrr,upward_volume_need_rte,upward_volume_rr,upward_weighted_average_price,upward_weighted_average_price_as,upward_weighted_average_price_margin,upward_weighted_average_price_mfrr,upward_weighted_average_price_rr,value,xb_balancing_exported_volume,xb_balancing_imported_volume
0,2014-02-07T18:00:00+01:00,2014-02-07T20:00:00+01:00,NaN,NaN,NaN,NaN,UPWARD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,DOWNGRADED,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2014-02-04T00:00:00+01:00,2014-02-04T03:00:00+01:00,NaN,NaN,NaN,NaN,UPWARD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,DOWNGRADED,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2014-02-03T08:00:00+01:00,2014-02-03T23:59:00+01:00,NaN,NaN,NaN,NaN,UPWARD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,DOWNGRADED,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2014-01-30T00:00:00+01:00,2014-01-30T01:00:00+01:00,NaN,NaN,NaN,NaN,UPWARD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,DOWNGRADED,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2014-01-29T14:30:00+01:00,2014-01-29T23:59:00+01:00,NaN,NaN,NaN,NaN,UPWARD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,DOWNGRADED,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [62]:
print doo.start_date
print doo.start_date.astype(np.datetime64)

In [37]:
df = pd.DataFrame(results).drop_duplicates()
df.set_index(["start_date","end_date"], inplace=True)
valueList = list(df.select_dtypes(include=["int","float","float64","long","object"]).columns)


stat = []

for res_name in df.resource.unique():
    sub_df = df[df.resource == res_name]
    
    for value_type in valueList:
        serie = sub_df[value_type].dropna()
        if not serie.empty:
            print value_type
            nth = pd.date_range(
                start=serie.index.get_level_values("start_date").astype(np.datetime64).min(), 
                end=serie.index.get_level_values("end_date").astype(np.datetime64).max(), 
                freq='30T')
        
            stat.append({
                "resource":res_name,
                "type": value_type,
                "début": serie.index.get_level_values("start_date").astype(np.datetime64).min(),
                "fin": serie.index.get_level_values("end_date").astype(np.datetime64).max(),
                "manquant": str(nth.shape[0]-serie.index.get_level_values("start_date").unique().shape[0])+" / "+str(nth.shape[0])
            })

do = pd.DataFrame(stat)
do.set_index(['resource','type'])

début   \
resource                type                                                                   
insufficients_offers    direction                                       2013-12-31 23:00:00    
                        nature                                          2013-12-31 23:00:00    
                        resource                                        2013-12-31 23:00:00    
                        type                                            2013-12-31 23:00:00    
peak_daily_margins      available_value                                 2014-01-01 10:00:00    
                        required_value                                  2014-01-01 10:00:00    
                        resource                                        2014-01-01 10:00:00    
                        type                                            2014-01-01 10:00:00    
imbalance_data          imbalance                                       2017-04-06 22:00:00    
                        missing_data_list                               2016-01-01 23:00:00    
                        negative_imbalance_settlement_price             2016-01-01 23:00:00    
                        positive_imbalance_settlement_price             2016-01-01 23:00:00    
                        resource                                        2016-01-01 23:00:00    
                        system_trend                                    2016-01-01 23:00:00    
prices                  downward_marginal_price                         2016-01-01 23:00:00    
                        downward_marginal_price_as                      2017-04-06 22:00:00    
                        downward_marginal_price_margin                  2017-04-06 22:00:00    
                        downward_weighted_average_price                 2016-01-01 23:00:00    
                        downward_weighted_average_price_as              2017-04-06 22:00:00    
                        downward_weighted_average_price_margin          2017-04-06 22:00:00    
                        downward_weighted_average_price_mfrr            2017-04-06 22:00:00    
                        downward_weighted_average_price_rr              2017-04-06 22:00:00    
                        resource                                        2016-01-01 23:00:00    
                        upward_marginal_price                           2016-01-01 23:00:00    
                        upward_marginal_price_as                        2017-04-06 22:00:00    
                        upward_marginal_price_margin                    2017-04-06 22:00:00    
                        upward_weighted_average_price                   2016-01-01 23:00:00    
                        upward_weighted_average_price_as                2017-04-06 22:00:00    
                        upward_weighted_average_price_margin            2017-04-06 22:00:00    
                        upward_weighted_average_price_mfrr              2017-04-06 22:00:00    
                        upward_weighted_average_price_rr                2017-04-06 22:00:00    
signals                 resource                                        2016-07-05 22:00:00    
                        type                                            2016-07-05 22:00:00    
                        value                                           2016-07-05 22:00:00    
procured_reserves       direction                                       2014-12-17 23:00:00    
                        price                                           2014-12-31 23:00:00    
                        resource                                        2014-12-17 23:00:00    
                        type                                            2014-12-17 23:00:00    
                        value                                           2014-12-17 23:00:00    
volumes_per_energy_type countertrading_xb_redispatching_exported_volume 2016-06-02 22:00:00    
                        countertrading_xb_redispatchin

In [11]:
dodo = pd.DataFrame(results)
dodo[dodo.resource == "accepted_offers"]

,available_value,countertrading_xb_redispatching_exported_volume,countertrading_xb_redispatching_imported_volume,deltap_volume,direction,downward_marginal_price,downward_marginal_price_as,downward_marginal_price_margin,downward_volume_afrr,downward_volume_fcr,downward_volume_mfrr,downward_volume_need_rte,downward_volume_rr,downward_weighted_average_price,downward_weighted_average_price_as,downward_weighted_average_price_margin,downward_weighted_average_price_mfrr,downward_weighted_average_price_rr,end_date,imbalance,...,system_trend,tso_mutual_emergency_exported_volume,tso_mutual_emergency_imported_volume,type,upward_marginal_price,upward_marginal_price_as,upward_marginal_price_margin,upward_volume_afrr,upward_volume_fcr,upward_volume_mfrr,upward_volume_need_rte,upward_volume_rr,upward_weighted_average_price,upward_weighted_average_price_as,upward_weighted_average_price_margin,upward_weighted_average_price_mfrr,upward_weighted_average_price_rr,value,xb_balancing_exported_volume,xb_balancing_imported_volume
83076,NaN,NaN,NaN,NaN,UP_DOWN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-12-17T00:30:00+01:00,NaN,...,NaN,NaN,NaN,FCR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,638,NaN,NaN
83077,NaN,NaN,NaN,NaN,UP_DOWN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-12-17T01:00:00+01:00,NaN,...,NaN,NaN,NaN,FCR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,704,NaN,NaN
83078,NaN,NaN,NaN,NaN,UP_DOWN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-12-17T01:30:00+01:00,NaN,...,NaN,NaN,NaN,FCR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,719,NaN,NaN
83079,NaN,NaN,NaN,NaN,UP_DOWN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-12-17T02:00:00+01:00,NaN,...,NaN,NaN,NaN,FCR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,715,NaN,NaN
83080,NaN,NaN,NaN,NaN,UP_DOWN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-12-17T02:30:00+01:00,NaN,...,NaN,NaN,NaN,FCR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,704,NaN,NaN
83081,NaN,NaN,NaN,NaN,UP_DOWN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-12-17T03:00:00+01:00,NaN,...,NaN,NaN,NaN,FCR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,723,NaN,NaN
83082,NaN,NaN,NaN,NaN,UP_DOWN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-12-17T03:30:00+01:00,NaN,...,NaN,NaN,NaN,FCR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,723,NaN,NaN
83083,NaN,NaN,NaN,NaN,UP_DOWN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-12-17T04:00:00+01:00,NaN,...,NaN,NaN,NaN,FCR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,721,NaN,NaN
83084,NaN,NaN,NaN,NaN,UP_DOWN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-12-17T04:30:00+01:00,NaN,...,NaN,NaN,NaN,FCR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,706,NaN,NaN
83085,NaN,NaN,NaN,NaN,UP_DOWN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-12-17T05:00:00+01:00,NaN,...,NaN,NaN,NaN,FCR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,733,NaN,NaN


In [83]:
writer = pd.ExcelWriter(".\Rapports_Etude_MA\Relation_Alert_Prix_MA\Results\Data_availability.xlsx", engine='xlsxwriter')

df.to_excel(writer,sheet_name="2016-2017",index=False, na_rep="-", encoding='utf-8')
# Close the Pandas Excel writer and output the Excel file.
writer.save()